<h1 align=center><font size = 5>Introduction to Deep Learning Peer Graded Assignment: Build a Regression Model in Keras</font></h1>

This project is the final assignement of the Introduction to Deep Learning Course which itself the 3rd course of the IBM AI Engineering professional track containing 6 courses.

# Preliminary

Installing Libraries (uncomment if necessary)

In [1]:
#pip install keras
#pip install tensorflow

Importing libraries

In [2]:
import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.utils import to_categorical 

import pandas as pd
import numpy as np
import sklearn as sk

Using TensorFlow backend.


Loading dataset

In [3]:
concrete_data = pd.read_csv('concrete_data.csv')
concrete_data.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age,Strength
0,540.0,0.0,0.0,162.0,2.5,1040.0,676.0,28,79.99
1,540.0,0.0,0.0,162.0,2.5,1055.0,676.0,28,61.89
2,332.5,142.5,0.0,228.0,0.0,932.0,594.0,270,40.27
3,332.5,142.5,0.0,228.0,0.0,932.0,594.0,365,41.05
4,198.6,132.4,0.0,192.0,0.0,978.4,825.5,360,44.30


Quick quality checks on data

In [4]:
concrete_data.describe()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age,Strength
count,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000
mean,281.167864,73.895825,54.188350,181.567282,6.204660,972.918932,773.580485,45.662136,35.817961
std,104.506364,86.279342,63.997004,21.354219,5.973841,77.753954,80.175980,63.169912,16.705742
min,102.000000,0.000000,0.000000,121.800000,0.000000,801.000000,594.000000,1.000000,2.330000
25%,192.375000,0.000000,0.000000,164.900000,0.000000,932.000000,730.950000,7.000000,23.710000
50%,272.900000,22.000000,0.000000,185.000000,6.400000,968.000000,779.500000,28.000000,34.445000
75%,350.000000,142.950000,118.300000,192.000000,10.200000,1029.400000,824.000000,56.000000,46.135000
max,540.000000,359.400000,200.100000,247.000000,32.200000,1145.000000,992.600000,365.000000,82.600000


In [5]:
concrete_data.isnull().sum()

Cement                0
Blast Furnace Slag    0
Fly Ash               0
Water                 0
Superplasticizer      0
Coarse Aggregate      0
Fine Aggregate        0
Age                   0
Strength              0
dtype: int64

# Part A - Build a Baseline Model

In [6]:
#Reusing code from one of the lab
def regression_model_custom(n_pred,hidden_layers=1, nodes=10):
### Customised the function so that I can build a model with more or less hidden layers and with more or less nodes.
    # create model
    model = Sequential()
    model.add(Dense(nodes, activation='relu', input_shape=(n_pred,))) #Fist Hidden Layer
    for i in range(2,hidden_layers+1):
        model.add(Dense(nodes, activation='relu')) #Other hidden layers if any
    model.add(Dense(1))
    
    # compile model
    model.compile(optimizer='adam', loss='mean_squared_error')
    return model

In [7]:
r_model=regression_model_custom(7)

## 1 - Split the data into training and testing set

In [8]:
from sklearn.model_selection import train_test_split

In [9]:
help(train_test_split)

Help on function train_test_split in module sklearn.model_selection._split:

train_test_split(*arrays, **options)
    Split arrays or matrices into random train and test subsets
    
    Quick utility that wraps input validation and
    ``next(ShuffleSplit().split(X, y))`` and application to input data
    into a single call for splitting (and optionally subsampling) data in a
    oneliner.
    
    Read more in the :ref:`User Guide <cross_validation>`.
    
    Parameters
    ----------
    *arrays : sequence of indexables with same length / shape[0]
        Allowed inputs are lists, numpy arrays, scipy-sparse
        matrices or pandas dataframes.
    
    test_size : float, int or None, optional (default=None)
        If float, should be between 0.0 and 1.0 and represent the proportion
        of the dataset to include in the test split. If int, represents the
        absolute number of test samples. If None, the value is set to the
        complement of the train size. If ``train_s

In [10]:
y=concrete_data['Strength']
X=concrete_data.drop(['Strength','Age'], axis=1)

In [11]:
#Randomly split the data into a training and test sets by holding 30% of the data for testing.
X_train_A, X_test_A, y_train_A, y_test_A=train_test_split(X,y,test_size=0.3)

In [13]:
X_train_A.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate
345,213.7,0.0,174.7,154.8,10.2,1053.5,776.4
569,381.4,0.0,0.0,185.7,0.0,1104.6,784.3
960,284.0,119.7,0.0,168.3,7.2,970.4,794.2
927,157.0,214.0,152.0,200.0,9.0,819.0,704.0
334,275.1,0.0,121.4,159.5,9.9,1053.6,777.5


## 2 - Training the model

In [14]:
help(keras.models.Sequential.fit) #Quick glance at Keras documentation to check how to not further split the data

Help on function fit in module keras.engine.training:

fit(self, x=None, y=None, batch_size=None, epochs=1, verbose=1, callbacks=None, validation_split=0.0, validation_data=None, shuffle=True, class_weight=None, sample_weight=None, initial_epoch=0, steps_per_epoch=None, validation_steps=None, validation_freq=1, max_queue_size=10, workers=1, use_multiprocessing=False, **kwargs)
    Trains the model for a fixed number of epochs (iterations on a dataset).
    
    # Arguments
        x: Input data. It could be:
            - A Numpy array (or array-like), or a list of arrays
              (in case the model has multiple inputs).
            - A dict mapping input names to the corresponding
              array/tensors, if the model has named inputs.
            - A generator or `keras.utils.Sequence` returning
              `(inputs, targets)` or `(inputs, targets, sample weights)`.
            - None (default) if feeding from framework-native
              tensors (e.g. TensorFlow data te

We see in the doc that if we do not specify a validation_split, the data won't be further splitted

In [15]:
#Train the model on the training data using 50 epochs.
r_model.fit(X_train_A, y_train_A, epochs=50, verbose=1)

Epoch 1/50
721/721 [==============================] - 0s 204us/step - loss: 123230.8837
Epoch 2/50
721/721 [==============================] - 0s 25us/step - loss: 41295.1320
Epoch 3/50
721/721 [==============================] - 0s 23us/step - loss: 10422.3174
Epoch 4/50
721/721 [==============================] - 0s 25us/step - loss: 4528.7443
Epoch 5/50
721/721 [==============================] - 0s 26us/step - loss: 4138.4475
Epoch 6/50
721/721 [==============================] - 0s 25us/step - loss: 3957.1796
Epoch 7/50
721/721 [==============================] - 0s 25us/step - loss: 3767.1078
Epoch 8/50
721/721 [==============================] - 0s 25us/step - loss: 3586.7508
Epoch 9/50
721/721 [==============================] - 0s 25us/step - loss: 3411.5020
Epoch 10/50
721/721 [==============================] - 0s 26us/step - loss: 3241.8995
Epoch 11/50
721/721 [==============================] - 0s 26us/step - loss: 3077.5418
Epoch 12/50
721/721 [==============================] - ETA

## 3 - Evaluating the model

In [16]:
#Evaluate the model on the test data and compute the mean squared error between the predicted concrete strength and the actual concrete strength
r_model.evaluate(X_test_A, y_test_A, verbose=1) #mean_squared_error using keras

309/309 [==============================] - 0s 74us/step


492.02376548989304

In [17]:

help(keras.models.Sequential.predict)

Help on function predict in module keras.engine.training:

predict(self, x, batch_size=None, verbose=0, steps=None, callbacks=None, max_queue_size=10, workers=1, use_multiprocessing=False)
    Generates output predictions for the input samples.
    
    Computation is done in batches.
    
    # Arguments
        x: Input data. It could be:
            - A Numpy array (or array-like), or a list of arrays
              (in case the model has multiple inputs).
            - A dict mapping input names to the corresponding
              array/tensors, if the model has named inputs.
            - A generator or `keras.utils.Sequence` returning
              `(inputs, targets)` or `(inputs, targets, sample weights)`.
            - None (default) if feeding from framework-native
              tensors (e.g. TensorFlow data tensors).
        batch_size: Integer or `None`.
            Number of samples per gradient update.
            If unspecified, `batch_size` will default to 32.
            

In [18]:
y_predicted_A=r_model.predict(X_test_A)
y_predicted_A[:5]

array([[35.551178 ],
       [41.087395 ],
       [44.214478 ],
       [14.505646 ],
       [-1.5174484]], dtype=float32)

In [19]:
from sklearn.metrics import mean_squared_error

In [20]:
MSE0_A=mean_squared_error(y_test_A,y_predicted_A) #mean_squared_error using Sciki-learn
MSE0_A

492.02376377906336

We do find the same mean squared error

## 4 - Repeating steps 1-3

In [22]:
error_list_A=[MSE0_A]

for i in range(50):
    print("Iteration #", i)
    X_train_A, X_test_A, y_train_A, y_test_A=train_test_split(X,y,test_size=0.3)
    r_model.fit(X_train_A, y_train_A, epochs=50, verbose=0)
    r_model.evaluate(X_test_A, y_test_A, verbose=0)
    y_predicted_A=r_model.predict(X_test_A)
    error_list_A.append(mean_squared_error(y_test_A,y_predicted_A))

Iteration # 0
Iteration # 1
Iteration # 2
Iteration # 3
Iteration # 4
Iteration # 5
Iteration # 6
Iteration # 7
Iteration # 8
Iteration # 9
Iteration # 10
Iteration # 11
Iteration # 12
Iteration # 13
Iteration # 14
Iteration # 15
Iteration # 16
Iteration # 17
Iteration # 18
Iteration # 19
Iteration # 20
Iteration # 21
Iteration # 22
Iteration # 23
Iteration # 24
Iteration # 25
Iteration # 26
Iteration # 27
Iteration # 28
Iteration # 29
Iteration # 30
Iteration # 31
Iteration # 32
Iteration # 33
Iteration # 34
Iteration # 35
Iteration # 36
Iteration # 37
Iteration # 38
Iteration # 39
Iteration # 40
Iteration # 41
Iteration # 42
Iteration # 43
Iteration # 44
Iteration # 45
Iteration # 46
Iteration # 47
Iteration # 48
Iteration # 49


In [23]:
error_list_A

[492.02376377906336,
 183.6263177775172,
 164.87005879996616,
 164.97663272406012,
 162.76349588002296,
 163.5653750641188,
 150.51765332569582,
 168.5082875761389,
 144.2888772282146,
 162.3766801359316,
 175.13219946403393,
 151.81946817940326,
 156.59481572433953,
 167.05323611877924,
 163.52698188801486,
 170.30816339466978,
 153.05908495704847,
 145.54182592395915,
 150.79624543331306,
 154.25718628653814,
 162.0934597835949,
 149.0377022756831,
 145.40211641657973,
 174.0040784482234,
 158.03388422309035,
 154.0537807212556,
 160.75651216469632,
 158.72336651349,
 134.72450199935034,
 147.1260346285353,
 134.24167272897498,
 169.90605653639116,
 165.62574960759576,
 151.50012042927204,
 140.38490886638817,
 160.9510955818208,
 154.102039596027,
 148.57567137410243,
 173.89898627759814,
 155.59150637934246,
 160.13208226503693,
 155.25543686449086,
 153.62161396483984,
 142.3908641369666,
 157.44643472388265,
 152.92027776686263,
 148.3013196841121,
 155.85212123974821,
 148.03482

## Reporting (statistical analysis)

In [24]:
from statistics import mean,pstdev

In [25]:
avg_MSE_A=mean(error_list_A)
avg_MSE_A

163.2892151255879

After 50 The average mean squared error is 160 

In [26]:
sigma_MSE_A=pstdev(error_list_A)
sigma_MSE_A

47.57406374405055

The population standard deviation is 10.8

# Part B - Normalize the data

In [27]:
y=concrete_data['Strength']
X=concrete_data.drop(['Strength','Age'], axis=1)
X.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate
0,540.0,0.0,0.0,162.0,2.5,1040.0,676.0
1,540.0,0.0,0.0,162.0,2.5,1055.0,676.0
2,332.5,142.5,0.0,228.0,0.0,932.0,594.0
3,332.5,142.5,0.0,228.0,0.0,932.0,594.0
4,198.6,132.4,0.0,192.0,0.0,978.4,825.5


In [28]:
X_norm=(X-X.mean())/X.std()
X_norm.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate
0,2.476712,-0.856472,-0.846733,-0.916319,-0.620147,0.862735,-1.217079
1,2.476712,-0.856472,-0.846733,-0.916319,-0.620147,1.055651,-1.217079
2,0.491187,0.795140,-0.846733,2.174405,-1.038638,-0.526262,-2.239829
3,0.491187,0.795140,-0.846733,2.174405,-1.038638,-0.526262,-2.239829
4,-0.790075,0.678079,-0.846733,0.488555,-1.038638,0.070492,0.647569


In [30]:
error_list_B=[]

for i in range(51):
    print("Iteration #", i)
    X_train_B, X_test_B, y_train_B, y_test_B=train_test_split(X_norm,y,test_size=0.3)
    r_model.fit(X_train_B, y_train_B, epochs=50, verbose=0)
    r_model.evaluate(X_test_B, y_test_B, verbose=0)
    y_predicted_B=r_model.predict(X_test_B)
    error_list_B.append(mean_squared_error(y_test_B,y_predicted_B))

Iteration # 0
Iteration # 1
Iteration # 2
Iteration # 3
Iteration # 4
Iteration # 5
Iteration # 6
Iteration # 7
Iteration # 8
Iteration # 9
Iteration # 10
Iteration # 11
Iteration # 12
Iteration # 13
Iteration # 14
Iteration # 15
Iteration # 16
Iteration # 17
Iteration # 18
Iteration # 19
Iteration # 20
Iteration # 21
Iteration # 22
Iteration # 23
Iteration # 24
Iteration # 25
Iteration # 26
Iteration # 27
Iteration # 28
Iteration # 29
Iteration # 30
Iteration # 31
Iteration # 32
Iteration # 33
Iteration # 34
Iteration # 35
Iteration # 36
Iteration # 37
Iteration # 38
Iteration # 39
Iteration # 40
Iteration # 41
Iteration # 42
Iteration # 43
Iteration # 44
Iteration # 45
Iteration # 46
Iteration # 47
Iteration # 48
Iteration # 49
Iteration # 50


In [31]:
error_list_B

[194.5912171700066,
 175.47130228652452,
 142.0881393507858,
 147.82215465911733,
 150.42861074655895,
 143.14307084194317,
 129.76002247965613,
 125.75819107277835,
 145.15327800505415,
 137.88786481770634,
 127.14788623171107,
 135.91366776311912,
 140.9438691671348,
 120.64103673128265,
 128.7849284924106,
 136.8461105673104,
 121.54766401419826,
 137.60814212066632,
 128.60995660097626,
 124.33931982335139,
 146.31026276920656,
 128.1390974379125,
 140.58295456859088,
 140.8423613578113,
 118.01504284038681,
 122.29316176464327,
 131.7703271946003,
 119.12577583906831,
 129.23566462189208,
 124.29829337310679,
 134.2814922347651,
 130.65894429740376,
 141.47844738043267,
 136.87155193689432,
 112.66480667175433,
 122.29235466672812,
 131.4545988534446,
 128.2743868803867,
 137.552711594681,
 118.10016885878018,
 129.4867117961278,
 130.7464291330025,
 133.3482625158875,
 129.24768572397207,
 118.96040948747915,
 121.54321710071268,
 125.14573511435844,
 126.60280145646628,
 116.093

In [32]:
avg_MSE_B=mean(error_list_B)
avg_MSE_B

133.09288586349766

In [33]:
sigma_MSE_B=pstdev(error_list_B)
sigma_MSE_B

13.7747180035397

Normalizing the data drove down the mean squared error. Normalizing the features probably leads to a better model by reducing the weight of certain features.

# Part C - Increase the number of epocs

In [34]:
#Train the model on the training data using 50 epochs.
r_model.fit(X_train_B, y_train_B, epochs=100, verbose=1)

Epoch 1/100
721/721 [==============================] - 0s 29us/step - loss: 115.6033
Epoch 2/100
721/721 [==============================] - 0s 27us/step - loss: 115.5585
Epoch 3/100
721/721 [==============================] - 0s 26us/step - loss: 115.6249
Epoch 4/100
721/721 [==============================] - 0s 26us/step - loss: 115.6566
Epoch 5/100
721/721 [==============================] - 0s 28us/step - loss: 115.6002
Epoch 6/100
721/721 [==============================] - 0s 26us/step - loss: 115.5182
Epoch 7/100
721/721 [==============================] - 0s 25us/step - loss: 115.5785
Epoch 8/100
721/721 [==============================] - 0s 26us/step - loss: 115.5491
Epoch 9/100
721/721 [==============================] - 0s 26us/step - loss: 115.5366
Epoch 10/100
721/721 [==============================] - 0s 28us/step - loss: 115.5472
Epoch 11/100
721/721 [==============================] - 0s 26us/step - loss: 115.5692
Epoch 12/100
721/721 [==============================] - ETA: 0s

721/721 [==============================] - 0s 26us/step - loss: 114.9593
Epoch 95/100
721/721 [==============================] - 0s 26us/step - loss: 115.0223
Epoch 96/100
721/721 [==============================] - 0s 25us/step - loss: 114.8746
Epoch 97/100
721/721 [==============================] - 0s 25us/step - loss: 114.9994
Epoch 98/100
721/721 [==============================] - 0s 24us/step - loss: 114.9524
Epoch 99/100
721/721 [==============================] - 0s 25us/step - loss: 114.9079
Epoch 100/100
721/721 [==============================] - 0s 24us/step - loss: 114.9682


In [35]:
error_list_C=[]

for i in range(51):
    print("Iteration #", i)
    X_train_B, X_test_B, y_train_B, y_test_B=train_test_split(X_norm,y,test_size=0.3)
    r_model.fit(X_train_B, y_train_B, epochs=100, verbose=0) #Notice the increase in the number of epochs
    r_model.evaluate(X_test_B, y_test_B, verbose=0)
    y_predicted_C=r_model.predict(X_test_B)
    error_list_C.append(mean_squared_error(y_test_B,y_predicted_C))

Iteration # 0
Iteration # 1
Iteration # 2
Iteration # 3
Iteration # 4
Iteration # 5
Iteration # 6
Iteration # 7
Iteration # 8
Iteration # 9
Iteration # 10
Iteration # 11
Iteration # 12
Iteration # 13
Iteration # 14
Iteration # 15
Iteration # 16
Iteration # 17
Iteration # 18
Iteration # 19
Iteration # 20
Iteration # 21
Iteration # 22
Iteration # 23
Iteration # 24
Iteration # 25
Iteration # 26
Iteration # 27
Iteration # 28
Iteration # 29
Iteration # 30
Iteration # 31
Iteration # 32
Iteration # 33
Iteration # 34
Iteration # 35
Iteration # 36
Iteration # 37
Iteration # 38
Iteration # 39
Iteration # 40
Iteration # 41
Iteration # 42
Iteration # 43
Iteration # 44
Iteration # 45
Iteration # 46
Iteration # 47
Iteration # 48
Iteration # 49
Iteration # 50


In [36]:
error_list_C

[130.28404182580329,
 137.40137468825898,
 132.6869539880737,
 137.28326542356263,
 136.82682981347494,
 117.46747378465352,
 121.43729774869647,
 143.00729086481064,
 127.40326043531542,
 125.8489151759688,
 130.73770781657402,
 137.20212687919624,
 126.47180847605057,
 112.92083761250338,
 130.62107058722503,
 134.5583725791574,
 136.11928021634316,
 124.07805955296976,
 121.33472886098455,
 128.22506072675407,
 129.65511734958855,
 133.38959388601418,
 121.39018534476452,
 123.20390197590905,
 125.0924076775945,
 124.21793782683116,
 119.23427091546917,
 130.447181462877,
 128.94431839606997,
 129.88875563378053,
 135.72312350109937,
 127.01097524363439,
 136.5958755204289,
 118.05346023528556,
 128.16143013756465,
 125.05299536899615,
 117.30532174476183,
 137.39138528414628,
 108.80964567321453,
 119.70594480023985,
 124.3901405272466,
 129.46391263282285,
 121.14769037628878,
 114.31336250363032,
 128.3714812857717,
 119.21689167088367,
 113.76114731093745,
 128.18135659189966,
 

In [38]:
avg_MSE_C=mean(error_list_C)
avg_MSE_C

126.95451640933885

In [39]:
sigma_MSE_C=pstdev(error_list_C)
sigma_MSE_C

7.459337795890611

Increasing the number of epoch also drove down the mean square error as the model becomes more accurate as it is trained longer.

# Part D - Increase the number of layer

In [40]:
r_model2=regression_model_custom(7,3,10)

In [41]:
error_list_D=[]
for i in range(51):
    print("Iteration #", i)
    X_train_B, X_test_B, y_train_B, y_test_B=train_test_split(X_norm,y,test_size=0.3)
    r_model2.fit(X_train_B, y_train_B, epochs=50, verbose=0) #Notice the increase in the number of epochs
    r_model2.evaluate(X_test_B, y_test_B, verbose=0)
    y_predicted_D=r_model.predict(X_test_B)
    error_list_D.append(mean_squared_error(y_test_B,y_predicted_D))

Iteration # 0
Iteration # 1
Iteration # 2
Iteration # 3
Iteration # 4
Iteration # 5
Iteration # 6
Iteration # 7
Iteration # 8
Iteration # 9
Iteration # 10
Iteration # 11
Iteration # 12
Iteration # 13
Iteration # 14
Iteration # 15
Iteration # 16
Iteration # 17
Iteration # 18
Iteration # 19
Iteration # 20
Iteration # 21
Iteration # 22
Iteration # 23
Iteration # 24
Iteration # 25
Iteration # 26
Iteration # 27
Iteration # 28
Iteration # 29
Iteration # 30
Iteration # 31
Iteration # 32
Iteration # 33
Iteration # 34
Iteration # 35
Iteration # 36
Iteration # 37
Iteration # 38
Iteration # 39
Iteration # 40
Iteration # 41
Iteration # 42
Iteration # 43
Iteration # 44
Iteration # 45
Iteration # 46
Iteration # 47
Iteration # 48
Iteration # 49
Iteration # 50


In [42]:
avg_MSE_D=mean(error_list_D)
avg_MSE_D

119.61728475650554

In [43]:
sigma_MSE_D=pstdev(error_list_D)
sigma_MSE_D

8.381524690789036

Finally, increasing the number of hidden layers also drove down the mean squared error as the model was gaining in complexity